### Bài toán

Các khó khăn gặp phải khi xây dựng mô hình phân loại văn bản tiếng Việt:

1. Việc tách từ tiếng Việt khá là phức tạp do có nhiều từ và cụm từ ghép lại với nhau, nhiều từ không có trong từ điển.
2. Việc tìm được bộ từ điển tương đối đầy đủ khá khó khăn do tiếng Việt có nhiều từ ngữ cả trong lĩnh vực chuyên ngành và nhiều từ ngữ không có trong từ điển. Hiện tại mới chỉ tìm được một số ít từ phổ biến có trong các từ điển mở.
3. Việc triển khai mô hình phân loại văn bản tiếng Việt khá là khó khăn do không có nhiều tài liệu tham khảo, đặc biệt là các tài liệu về xử lý ngôn ngữ tự nhiên tiếng Việt.
4. Việc cài đặt môi trường lập trình cho Tensorflow cũng như pytorch gặp nhiều khó khăn khi cài đặt trên Windows, đặc biệt là khi sử dụng GPU.
5. Việc chuẩn hoá văn bản tiếng Việt gặp rất nhiều thách thức về:
    - Cách mã hoá văn bản Unicode
    - Cách bỏ dấu tiếng Việt
    - Xử lý sai chính tả
    - Xử lý từ viết tắt
    - Xử lý các từ ngữ không có trong từ điển
7. Việc tách từ cũng gặp rất nhiều bất lợi
    - Từ điển càng lớn thì việc tách từ càng chậm, đã thử nghiệm và tách từ mất 30p cho tập train.
    - Tokenization phải đào tạo mô hình để thực hiện mới đem lại hiệu quả tốt. Đã thử nghiệm với các thư viện như pyvi, underthesea, vncorenlp, ... nhưng kết quả không tốt. Chậm quá
6. Hiện chưa có bộ StopWords tiếng Việt tương đối đầy đủ, nên việc xử lý StopWords cũng gặp nhiều khó khăn do đa phần là tự tạo không theo một tiêu chuẩn nào cả.

- Input: một đoạn văn bản
- Output: một trong các nhãn sau: 'XH':'Chinh tri Xa hoi', 
            'DS':'Doi song', 
            'KH':'Khoa hoc', 
            'KD':'Kinh doanh', 
            'PL':'Phap luat', 
            'SK':'Suc khoe', 
            'TG':'The gioi', 
            'TT':'The thao', 
            'VH':'Van hoa', 
            'VT':'Vi tinh',



### Thu thập dữ liệu


Dữ liệu được lấy dừ https://github.com/duyvuleo/VNTC/tree/master/Data/10Topics/Ver1.1 với thông tin như sau:

***Train***
|Topic |	Topic ID |	#files |
|--|--|--|
| Chinh tri Xa hoi	| XH |	5219 |
| Doi song	| DS |		3159 |
| Khoa hoc	| KH |		1820 |
| Kinh doanh	| KD |		2552 |
| Phap luat	| PL |		3868 |
| Suc khoe	| SK |		3384 |
| The gioi	| TG |		2898 |
| The thao	| TT |		5298 |
| Van hoa 	| VH |		3080 |
| Vi tinh		| VT |		2481 |

Total				33759

***Test***
|Topic |	Topic ID |	#files |
|--|--|--|
| Chinh tri Xa hoi	| XH |	7567 |
| Doi song	| DS |		2036 |
| Khoa hoc	| KH |		2096 |
| Kinh doanh	| KD |		5276 |
| Phap luat	| PL |		3788 |
| Suc khoe	| SK |		5417 |
| The gioi	| TG |		6716 |
| The thao	| TT |		6667 |
| Van hoa		| VH |		6250 |
| Vi tinh		| VT |		4560 |

Total				50373

Tên file được đặt cào từ:

+ DS_VNE_(...) : VnExpress news agency (http://vnexpress.net/)
+ DS_TT_(...):  Youth news agency (http://tuoitre.vn/)
+ DS_TN_(...): Thanh Nien news agency (http://thanhnien.vn/)
+ DS_NLD_(...): Nguoi Lao Dong news agency (http://nld.com.vn/)

File zip chứa toàn bộ file, tên mỗi file là Nhãn_Báo_ (STT).txt VD: XH_NLD_ (3675).txt

In [82]:
import zipfile

def extract_data_from_zip(zip_file_path):
    result = []
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        file_list = zip_ref.namelist()  # Lấy danh sách tên các file trong zip

        for file_name in file_list:
            if file_name.endswith('.txt'):  # Chỉ xử lý các file có đuôi .txt
                with zip_ref.open(file_name) as file:
                    data = file.read().decode('utf-16-le')  # Nó được mã hoá bằng utf-16-le
                    # nội dung file có dạng \ufeff<content>, đôi khi có dấu cách ở đầu và cuối
                    content = data[1:].strip()
                    words = content.split(' ')
                    content = ' '.join(words)#(words[:min(len(words), 20)])
                    label = file_name.split('/')[2].split('_')[0]
                    result.append([content, label])
    return result

In [83]:
train_data = extract_data_from_zip('data/Train_Full.zip')
len(train_data)

33759

In [84]:
test_data = extract_data_from_zip('data/Test_Full.zip')
len(test_data)

50373

In [85]:
labels = [x[1] for x in train_data]
labels = set(labels)
labels

{'DS', 'KD', 'KH', 'PL', 'SK', 'TG', 'TT', 'VH', 'VT', 'XH'}

In [86]:
label_dict={'XH':(0,'Chinh tri Xa hoi'), 
            'DS':(1,'Doi song'), 
            'KH':(2,'Khoa hoc'), 
            'KD':(3,'Kinh doanh'), 
            'PL':(4,'Phap luat'), 
            'SK':(5,'Suc khoe'), 
            'TG':(6,'The gioi'), 
            'TT':(7,'The thao'), 
            'VH':(8,'Van hoa'), 
            'VT':(9,'Vi tinh') }
print(*[label_dict[x][1] for x in labels], sep = ', ')

Doi song, Chinh tri Xa hoi, Van hoa, Vi tinh, The gioi, Khoa hoc, The thao, Phap luat, Suc khoe, Kinh doanh


### Tiền xử lý dữ liệu

#### Xoá HTML

Dữ liệu được thu thập từ các website đôi khi vẫn còn sót lại các đoạn mã HTML. Các mã HTML code này là rác, chẳng những không có tác dụng cho việc phân loại mà còn làm kết quả phân loại văn bản bị kém đi. Do đó, cần phải loại bỏ các đoạn mã HTML này.

In [87]:
import re
def remove_html(txt):
    return re.sub(r'<[^>]*>', ' ', txt)

txt = "<p class=\"par\">This is an example</p>nè"
remove_html(txt)

' This is an example nè'

#### Chuẩn hoá Tiếng Việt

- **Chuẩn hoá Unicode**: Hiện nay, có 2 loại mã Unicode được sử dụng phổ biến, Unicode tổ hợp và Unicode dựng sẵn. Hướng xử lý: Đưa về 1 chuẩn Unicode dựng sẵn (thằng này phổ biến hơn)
- **Chuẩn hoán cách bỏ dấu**: Chuyển câu văn về cách gõ dấu kiểu cũ: dùng òa úy thay oà uý

In [88]:
import re

uniChars = "àáảãạâầấẩẫậăằắẳẵặèéẻẽẹêềếểễệđìíỉĩịòóỏõọôồốổỗộơờớởỡợùúủũụưừứửữựỳýỷỹỵÀÁẢÃẠÂẦẤẨẪẬĂẰẮẲẴẶÈÉẺẼẸÊỀẾỂỄỆĐÌÍỈĨỊÒÓỎÕỌÔỒỐỔỖỘƠỜỚỞỠỢÙÚỦŨỤƯỪỨỬỮỰỲÝỶỸỴÂĂĐÔƠƯ"
unsignChars = "aaaaaaaaaaaaaaaaaeeeeeeeeeeediiiiiooooooooooooooooouuuuuuuuuuuyyyyyAAAAAAAAAAAAAAAAAEEEEEEEEEEEDIIIOOOOOOOOOOOOOOOOOOOUUUUUUUUUUUYYYYYAADOOU"


def loaddicchar():
    dic = {}
    char1252 = 'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ'.split(
        '|')
    charutf8 = "à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ".split(
        '|')
    for i in range(len(char1252)):
        dic[char1252[i]] = charutf8[i]
    return dic

dicchar = loaddicchar()

def convert_unicode(txt):
    return re.sub(
        r'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ',
        lambda x: dicchar[x.group()], txt)
    
dict_map = {"òa": "oà", "Òa": "Oà", "ÒA": "OÀ", "óa": "oá", "Óa": "Oá", "ÓA": "OÁ", "ỏa": "oả", "Ỏa": "Oả", "ỎA": "OẢ", "õa": "oã", "Õa": "Oã", "ÕA": "OÃ", "ọa": "oạ", "Ọa": "Oạ", "ỌA": "OẠ", "òe": "oè", "Òe": "Oè", "ÒE": "OÈ", "óe": "oé", "Óe": "Oé", "ÓE": "OÉ", "ỏe": "oẻ", "Ỏe": "Oẻ", "ỎE": "OẺ", "õe": "oẽ", "Õe": "Oẽ", "ÕE": "OẼ", "ọe": "oẹ", "Ọe": "Oẹ", "ỌE": "OẸ", "ùy": "uỳ", "Ùy": "Uỳ", "ÙY": "UỲ", "úy": "uý", "Úy": "Uý", "ÚY": "UÝ", "ủy": "uỷ", "Ủy": "Uỷ", "ỦY": "UỶ", "ũy": "uỹ", "Ũy": "Uỹ", "ŨY": "UỸ", "ụy": "uỵ", "Ụy": "Uỵ", "ỤY": "UỴ",}

def replace_all(text, dict_map):
    for i, j in dict_map.items():
        text = text.replace(i, j)
    return text
def vietnameseTextNormalizer(sentence):
    import unicodedata    
    return replace_all(unicodedata.normalize('NFC', convert_unicode(sentence)), dict_map)

str_utf8 = 'Anh Hòa, đang làm gì chị Thúy vậy, ăn qụyt phải không?' # Unicode (dựng sẵn - dấu theo ký tự)
str_utf8_2 = 'Anh Hoà, đang làm gì chị Thuý vậy, ăn quỵt phải không?' # Unicode (dựng sẵn - dấu theo ký tự)
str_com = 'Anh Hòa, đang làm gì chị Thúy vậy, ăn qụyt phải không?'  # Unicode composite (tổ hợp - dấu riêng)
str_1252 = 'Anh Hòa, đang làm gì chị Thúy vậy, ăn qụyt phải không?' # Windows-1252 = Latin-1
print(str_utf8 == str_utf8_2, str_utf8 == str_com, str_utf8 == str_1252, str_com == str_1252)
str_utf8 = vietnameseTextNormalizer(str_utf8)
str_utf8 = vietnameseTextNormalizer(str_utf8_2)
str_com = vietnameseTextNormalizer(str_com)
str_1252 = vietnameseTextNormalizer(str_1252)
print(str_utf8 == str_utf8_2, str_utf8 == str_com, str_utf8 == str_1252, str_com == str_1252)

False False False True
True True True True


#### Tách từ

File từ điển các từ và cụm từ được tổng hợp từ các nguồn sau:
- Viet74K.txt: https://github.com/undertheseanlp/underthesea/tree/main/underthesea/corpus/data
- words.txt: https://github.com/undertheseanlp/underthesea/tree/main/datasets/DI_Vietnamese-UVD/corpus/dictionary
- vi-vocab: https://github.com/vncorenlp/VnCoreNLP/tree/master/models/wordsegmenter
- Thư mục Words - Danh mục từ của wordnet: https://github.com/zeloru/vietnamese-wordnet/tree/master

Sau đó được xử lý để tạo thành 1 file từ điển duy nhất tên là [dic3.txt](data/dic3.txt) nhờ code từ: [Create_Data.ipynb](Create_Data.ipynb)

In [89]:
from collections import defaultdict
import re

def syllablize(sentence): # Tách âm tiết cho một câu tiếng Việt
    word = '\w+'
    non_word = '[^\w\s]'
    digits = '\d+([\.,_]\d+)+'
    
    patterns = []
    patterns.extend([word, non_word, digits])
    patterns = f"({'|'.join(patterns)})"
    
    tokens = re.findall(patterns, sentence, re.UNICODE)
    return [token[0] for token in tokens]

# Tải từ trong vi-vocab.txt
with open('data/dic3.txt', encoding='utf8') as f:
    vocab = f.read().split('\n')
# Xây dựng từ điển vocabs theo độ dài từ
vocabs = defaultdict(dict) # dùng list chậm hơn so với dict khoảng 25 lần
for word in vocab:
    vocabs[len(word.split())][word.lower()] = True

print('Số lượng từ ghép và cụm từ trong vocab:', len(vocab))
print('Số bộ vocab phân theo độ dài:', len(vocabs))

def longest_matching(sentence, vocabs):
  words = syllablize(sentence) # tách âm tiết cho câu
  result = []
  i = len(words)-1 # index của từ hiện tại
  while i > -1: 
    word = '' 
    # tìm kiếm trong từ điển theo chiều dài của từ ưu tiên từ dài trước
    for j in range(max(i-8,0), i+1):
      ls_word = words[j:i+1]
      word = ' '.join(ls_word)
      # xem thử có trong từ điển không
      if word.lower() in vocabs.get(i-j+1, []):
        i = j
        break
    result.append(word)
    i-=1
  return result[::-1] # return the final list

def tokenize_sentences(sentence):
    return ' '.join([x.replace(' ','_') for x in longest_matching(sentence, vocabs)])

tokenize_sentences('nhưng sự thực hiện vẫn còn chưa phù hợp')

Số lượng từ ghép và cụm từ trong vocab: 112343
Số bộ vocab phân theo độ dài: 20


'nhưng sự_thực_hiện vẫn_còn chưa phù_hợp'

#### Hoàn thiện tiền xử lý

In [90]:
def text_preprocess(document):
    # xóa html code
    document = remove_html(document)
    # chuẩn hóa unicode
    document = convert_unicode(document)
    # chuẩn hóa cách gõ dấu tiếng Việt
    document = vietnameseTextNormalizer(document)
    # tách từ
    document = tokenize_sentences(document)
    # đưa về lower
    document = document.lower()
    # xóa các ký tự không cần thiết
    document = re.sub(r'[^\s\wáàảãạăắằẳẵặâấầẩẫậéèẻẽẹêếềểễệóòỏõọôốồổỗộơớờởỡợíìỉĩịúùủũụưứừửữựýỳỷỹỵđ_]',' ',document)
    # xóa khoảng trắng thừa
    document = re.sub(r'\s+', ' ', document).strip()
    return document

text = text_preprocess('<p class=\"par\">Têu đề bài báo:</p>Có vẻ như Anh Hòa, đang làm gì đó với chị Thúy vậy, có thể là ăn qụyt có phải không?')
text

'têu đề bài_báo có_vẻ_như anh hoà đang làm_gì đó với chị thuý vậy có_thể_là ăn_quỵt có_phải không'

#### Loại bỏ stopword

In [91]:
with open('data/stopwords-nlp-vi.txt', encoding='utf8') as f:
    stopword = f.read().replace(' ','_').split('\n')

stopword = set(stopword)

def remove_stopwords(line):
    words = []
    for word in line.strip().split():
        if word not in stopword:
            words.append(word)
    return ' '.join(words)

text = remove_stopwords(text)
text

'têu đề bài_báo có_vẻ_như hoà thuý có_thể_là ăn_quỵt'

### Xây dựng mô hình phân loại văn bản

In [92]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

if torch.cuda.is_available():
    print("PyTorch is using GPU.")
    device = torch.device("cuda")  # Chọn GPU làm thiết bị tính toán
    print(f"Current GPU device: {torch.cuda.current_device()}")
    print(f"Number of available GPUs: {torch.cuda.device_count()}")
else:
    print("PyTorch is using CPU.")
    device = torch.device("cpu")  # Chọn CPU là thiết bị tính toán


PyTorch is using GPU.
Current GPU device: 0
Number of available GPUs: 1


Trong mô hình phân loại văn bản thì `vocab_size`, `embedding_dim`, `hidden_size`, và `num_classes` là các thông số quan trọng cần được xác định trước khi xây dựng mô hình:

1. `vocab_size` là kích thước của từ điển (vocabulary) trong mô hình phân loại văn bản. Nó đại diện cho số lượng từ duy nhất có trong tập dữ liệu huấn luyện. Khi xây dựng mô hình, mỗi từ sẽ được biểu diễn bằng một chỉ số số nguyên từ 0 đến `vocab_size - 1`. `vocab_size` có thể được tính toán tự động bằng cách đếm từ.

Để tính toán `vocab_size`, bạn có thể xây dựng từ điển từ các từ trong tập dữ liệu huấn luyện và đếm số lượng từ duy nhất. Một cách đơn giản để làm điều này là sử dụng một bộ đếm từ (word counter) để đếm số lần xuất hiện của mỗi từ trong tập dữ liệu và sau đó lấy độ dài của từ điển để có `vocab_size`.

2. `embedding_dim`: Đây là số chiều của không gian nhúng (embedding space) trong mô hình. Trong quá trình huấn luyện, các từ trong từ điển sẽ được biểu diễn bằng các vectơ có kích thước `embedding_dim`. Số chiều này cần được chọn sao cho đủ lớn để mô hình có thể học được các đặc trưng quan trọng của văn bản, nhưng cũng không quá lớn để tránh tăng quá nhiều tham số và tốn thời gian huấn luyện.Thông thường, kích thước không gian nhúng từ 100 đến 300 chiều đã được sử dụng hiệu quả trong nhiều nhiệm vụ phân loại văn bản.

3. `hidden_size`: Đây là số lượng đơn vị ẩn trong mạng LSTM (Long Short-Term Memory) hoặc các mạng RNN (Recurrent Neural Network) khác. `hidden_size` ảnh hưởng đến khả năng mô hình học các mẫu dữ liệu phức tạp. Nếu `hidden_size` lớn, mô hình có khả năng học các mẫu phức tạp hơn, nhưng đồng thời tăng cường độ phức tạp của mô hình và thời gian huấn luyện. Giá trị thông thường cho `hidden_size` trong mạng LSTM hoặc RNN là 100, 200 hoặc 300. Tuy nhiên, nếu tập dữ liệu lớn hoặc bài toán phân loại phức tạp hơn, có thể cần tăng giá trị này để mô hình có khả năng học mẫu phức tạp hơn.

4. `num_classes`: Đây là số lượng lớp trong bài toán phân loại. Đối với bài toán phân loại văn bản, `num_classes` sẽ là số lượng nhãn khác nhau mà chúng ta muốn mô hình phân loại các văn bản vào. Ví dụ, nếu ta có 3 nhãn: "positive", "negative", và "neutral", thì `num_classes` sẽ là 3.

Việc lựa chọn các giá trị tối ưu cho `embedding_dim`, `hidden_size`, và `num_classes` phụ thuộc vào bài toán cụ thể và dữ liệu đang làm việc. Thông thường, các giá trị này được chọn dựa trên kinh nghiệm thực tế và thử nghiệm. Trong quá trình huấn luyện, bạn có thể điều chỉnh các giá trị này và theo dõi hiệu suất của mô hình để tìm ra các giá trị phù hợp nhất.

In [93]:
def getPreprocessed_text(text):
    text = text_preprocess(text)
    text = remove_stopwords(text)
    return text

In [95]:
from tqdm import tqdm
# Xử lý dữ liệu train cho chuẩn đã
for i in tqdm(range(len(train_data))):
    train_data[i][0] = getPreprocessed_text(train_data[i][0])
## loại bỏ các dòng trống
train_data = [x for x in train_data if x[0] != '']

100%|██████████| 33759/33759 [03:34<00:00, 157.67it/s]


In [96]:
from tqdm import tqdm
# Xử lý dữ liệu Test cho chuẩn đã
for i in tqdm(range(len(test_data))):
    test_data[i][0] = getPreprocessed_text(test_data[i][0])
## loại bỏ các dòng trống
test_data = [x for x in test_data if x[0] != '']

100%|██████████| 50373/50373 [05:42<00:00, 146.96it/s]


In [97]:
def create_vocab_dict(data):
    word_counts = defaultdict(int)
    for text, label in data:
        for word in text.split():
            word_counts[word] += 1

    sorted_words = sorted(word_counts.items(), key=lambda x: x[1], reverse=True)
    vocab_dict = {}
    for i, (word, count) in enumerate(sorted_words):
        vocab_dict[word] = i + 1
    
    vocab_dict['<unk>'] = 0
    return vocab_dict

vocab_dict = create_vocab_dict(train_data)
print(dict(list(vocab_dict.items())[80:100]))

{'trần': 81, 'ngành': 82, 'trung_quốc': 83, 'nam': 84, 'phát_hiện': 85, 'bệnh': 86, '2006': 87, 'sản_phẩm': 88, '18': 89, 'dịch_vụ': 90, 'bóng_đá': 91, 'tiền_đạo': 92, 'chồng': 93, 'tuần': 94, 'yêu': 95, 'hệ_thống': 96, 'quốc_tế': 97, 'khi_đó': 98, 'người_ta': 99, 'người_dân': 100}


In [98]:
def sentence_to_num(sentence, vocab):
    tokens = sentence.split()
    sentence_indexes = [vocab.get(token, vocab['<unk>']) for token in tokens]
    return torch.tensor(sentence_indexes)

sentence_to_num(train_data[2][0], vocab_dict)

tensor([ 1002,  1043,   134,   702,   441,    11,    22,  1043,  1904,   921,
         9653,   260,  1085,   145,  2208,   132,  1435,   751,  7363,    61,
        23083,  1052, 32477,     9,   361,  1043, 32478,  3181,   134,   702,
          441, 80153, 80154, 31077, 31077,  2540,   981,  3372, 40118,    24,
         7018,  3302,    71,   825,   145,   702, 80155,  2651,   145,  1920,
          512,   294,  1370,  4178,  1043,  6534,  3141,  1043,  3759,   490,
          134,   702, 14877,     2, 12302,  3337,    11,    22])

In [99]:
vocab_size = len(vocab_dict)
embedding_dim = 100
hidden_size = 128
num_classes = len(labels)

print(vocab_size, num_classes)

122699 10


In [100]:
import torch.nn.functional as F
# Lớp tập dữ liệu tùy chỉnh
class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data
        
    def __getitem__(self, index):
        text, label = self.data[index]
        x = sentence_to_num(text, vocab_dict)
        index = label_dict[label][0]
        y = F.one_hot(torch.tensor(index), num_classes=num_classes).float()
        return x, y 
    
    def __len__(self):
        return len(self.data)    

In [101]:
import torch.nn as nn
import torch.nn.functional as F

class TextClassifier(nn.Module):
    def __init__(self, num_classes, vocab_size, embedding_dim, hidden_size):
        super(TextClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
        
    def forward(self, inputs, lengths):
        embedded = self.embedding(inputs)
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, lengths, batch_first=True, enforce_sorted=False)
        packed_output, (h_n, c_n) = self.lstm(packed_embedded)
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output, batch_first=True)
        last_output = torch.zeros(output.size(0), output.size(2)).to(inputs.device)
        for i in range(len(output)):
            last_output[i] = output[i, lengths[i]-1, :]
        logits = self.fc(last_output)
        return logits

In [104]:
# Cấu hình
batch_size = 100
epochs = 10
learning_rate = 0.001

# Chuẩn bị dữ liệu
train_dataset = CustomDataset(train_data)
test_dataset = CustomDataset(test_data)

from torch.nn.utils.rnn import pack_sequence

def collate_fn(batch):
    # split the batch into inputs and labels
    inputs, labels = zip(*batch)

    # get the lengths of the inputs
    lengths = [len(sample) for sample in inputs]

    # convert inputs to a PyTorch tensor
    input_tensor = torch.zeros(len(inputs), max(lengths)).long()
    for i, sample in enumerate(inputs):
        input_tensor[i, :lengths[i]] = sample.clone().detach() # tạo một bản sao của tensor gốc mà không bị tính toán gradient 
    # convert the labels to a PyTorch tensor
    label_tensor = torch.stack(labels)
    # return a tuple of the inputs, the lengths, and the labels
    return input_tensor, lengths, label_tensor

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

# Khởi tạo mô hình và tối ưu hóa
model = TextClassifier(num_classes, vocab_size, embedding_dim, hidden_size).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [105]:
# Huấn luyện
loss_values = []  # Khởi tạo danh sách để lưu trữ giá trị loss
model.train()
for epoch in tqdm(range(epochs)):
    for inputs, lengths, labels in train_loader:

        inputs = inputs.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs, lengths)
        loss = criterion(outputs, labels)
        # print(outputs[:10], labels[:10])
        # print(2/0)
        loss.backward()
        optimizer.step()
        loss_values.append(loss.item())  # Lưu trữ giá trị loss vào danh sách

100%|██████████| 10/10 [29:14<00:00, 175.44s/it]


In [107]:
loss_values[::-1]

[0.22966578602790833,
 0.4606103301048279,
 0.07397925108671188,
 0.28385111689567566,
 0.11748090386390686,
 0.25248366594314575,
 0.2559695839881897,
 0.1943567991256714,
 0.12524816393852234,
 0.1388106793165207,
 0.1409941017627716,
 0.254610538482666,
 0.13845297694206238,
 0.10281648486852646,
 0.13820473849773407,
 0.17204749584197998,
 0.06730099767446518,
 0.23571205139160156,
 0.23550480604171753,
 0.2207297533750534,
 0.22337596118450165,
 0.10490860044956207,
 0.22253280878067017,
 0.1981249898672104,
 0.29247432947158813,
 0.13204728066921234,
 0.113082654774189,
 0.156915083527565,
 0.1889570951461792,
 0.31750839948654175,
 0.11769203096628189,
 0.23477177321910858,
 0.13610681891441345,
 0.15921755135059357,
 0.13357225060462952,
 0.18089908361434937,
 0.23576369881629944,
 0.15968750417232513,
 0.2472904473543167,
 0.19624030590057373,
 0.1223946139216423,
 0.18201181292533875,
 0.2142481952905655,
 0.2493230253458023,
 0.147587388753891,
 0.2768251597881317,
 0.221898

In [ ]:
# import matplotlib.pyplot as plt
# plt.plot(loss_values)  # Vẽ đồ thị loss
# plt.xlabel('Iteration')
# plt.ylabel('Loss')
# plt.show()

In [109]:
# Đánh giá mô hình
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for inputs, lengths, labels in test_loader:
        
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs, lengths)
        _, predicted = torch.max(outputs.data, 1)
        _, lslabels = torch.max(labels, 1)
        total += lslabels.size(0)
        correct += (predicted == lslabels).sum().item()
    
    accuracy = 100 * correct / total
    print(f"Accuracy: {accuracy}%")

Accuracy: 85.82573997975106%


In [129]:
# Hàm trả về tên nhãn dự đoán
def predict(text):
    model.eval()
    with torch.no_grad():
        inputs = sentence_to_num(text, vocab_dict).unsqueeze(0).to(device)
        lengths = [inputs.size(1)]
        outputs = model(inputs, lengths)
        _, predicted = torch.max(outputs.data, 1)
        result = [label_dict[x][1] for x in label_dict if label_dict[x][0] == predicted.item()]
        return result[0]

test = test_data[4][0]
print(predict(test))
print(test)

Kinh doanh
dự_trữ ngoại_hối đang_chảy mỹ eu ngân_hàng trung_ương khoản dự_trữ ngoại_hối euro động_thái gây_khó_khăn tổng_thống_mỹ george bush giảm_bớt thâm_hụt tài_khoản_vãng_lai cân_đối nghiêm_trọng ngân_hàng trung_ương cấp_tiền_cho khoản thâm_hụt tài_khoản_vãng_lai mỹ mức_độ nhà_trắng dựa_vào nguồn_tài_chính mức_độ 65 quan_chức đứng_đầu ngân_hàng trung_ương giấu_tên trả_lời tra_của central banking publication công_ty thống_kê tài_chính mỹ báo_cáo điều_tra 2 3 65 quan_chức ngân_hàng dự_trữ ngoại_hối bằng_đồng euro tháng_11 2002 khoản dự_trữ ngoại_hối chủ_yếu bằng_đồng usd khoản dự_trữ chủ_yếu euro diễn_ra dự 2 mặc_dù kết_quả dẫn_đến mất_giá 2 tiền_tệ mức_độ báo_cáo kết_quả điều_tra euro ngân_hàng trung_ương cao_hơn mỹ_kim tư_cách phương_tiện dự_trữ ngoại_hối tỷ_giá gần_đây_nhất new york 1 euro đổi 1 3063 usd giám_đốc ngân_hàng trung_ương nhận_định thị_trường_tiền_tệ thị_trường mua_bán giấy_chứng_nhận nợ khu_vực hấp_dẫn khoản đầu_tư chẳng kém mỹ tổng_cộng mỹ_kim chứng_khoán có_giá usd 

In [130]:
# Lưu mô hình
torch.save(model.state_dict(), 'model.pth')

#### Xây dựng tập dữ liệu huấn luyện và kiểm thử

#### Xây dựng mô hình phân loại

### Đánh giá mô hình phân loại văn bản

### Tài liệu tham khảo